# Importation des données
[To Do : continuer de décrire origine et format des données]

Les données sont localisées dans des fichiers XML, chaque fichier correspondant à une demande de brevets. Ces fichiers sont directement téléchargeables depuis le serveur FTP de l'INPI, accessible via FileZilla.

Pour rendre notre projet reproductible au maximum, bien que les données ne puissent pas être directement téléchargées par un utilisateur non autorisé par l'INPI, nous avons stocké les fichiers .zip tels que nous les avons initialement téléchargés dans le bucket diffusion de l'un de nous. 

Concernant leur format, ces fichiers compressés contiennent chacun les informations sur toutes les demandes de brevets déposés auprès de l'INPI chaque semaine. Il y a donc 52 dossiers par année, un pour chaque semaine. Dans ces dossiers, se trouvent des informations dont nous n'avons pas besoin (ex: des schémas d'invention). Nous n'utiliserons que les données présentes dans des fichiers XML, chacun de ces fichiers contenant de nombreuses informations sur une demande de brevet déposée dans la semaine concernée.

Le code suivant se charge de décompresser ces fichiers, d'identifier les fichiers XML au sein de leurs arborescences et d'en extraire toutes les informations.

In [22]:
import os
import s3fs
import pandas as pd

from scripts import importation

In [ ]:
# Connexion au bucket S3 (MinIO)
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

ROOT_S3_PATH = "mvallat/diffusion/projet"

In [1]:
# NB : ce code tourne 2-3min
data_brevets = process_all_years_s3(fs, ROOT_S3_PATH);
data_brevets.head(2)

NameError: name 'process_all_years_s3' is not defined

In [32]:
# Exportation bucket S3
MY_BUCKET = "mvallat/diffusion"
FILE_PATH_OUT_S3 = f"{MY_BUCKET}/projet/data_brevets.parquet"

with fs.open(FILE_PATH_OUT_S3, 'wb') as file_out:
    data_brevets.to_parquet(file_out)

Notre base de données est maintenant stockée dans une bucket diffusion sur le ssp cloud, ce qui nous permet de ne pas avoir à repartir des fichiers compressés initiaux. 

**Pour importer directement les données sans passer par les étapes précédentes, on pourra utiliser le code suivant :**

In [33]:
# Importation bucket S3
MY_BUCKET = "mvallat/diffusion"
FILE_PATH_S3 = f"{MY_BUCKET}/projet/data_brevets.parquet"

with fs.open(FILE_PATH_S3, 'rb') as file_in:
    data_brevets = pd.read_parquet(file_in)

data_brevets.head(2)

,family_id,doc-number,kind,country,date-produced,status,publication_country,publication_doc-number,publication_date,publication_bopinum,...,citation_2_date,citation_3_type,citation_3_text,citation_3_country,citation_3_doc-number,citation_3_date,last-fee-payement,next-fee-payement,date-search-completed,year
0,1800103546,3098104,A1,FR,20210108,NEW,FR,3098104,20210108,2021-01,...,NA,NA,NA,NA,NA,NA,NA,20210802,NA,2021
1,1800103291,3098134,A1,FR,20210108,NEW,FR,3098134,20210108,2021-01,...,20151126,patcit,GB 2 545 190 A (QUANTUM CHEMICAL TECH PTE\nLT...,GB,GB-2545190-A,20170614,20200731,20210802,20210108,2021
